Model ilerlemesi eğitim sırasında ve sonrasında kaydedilebilir. Bu, bir modelin kaldığı yerden devam edebileceği ve uzun eğitim sürelerinden kaçınabileceği anlamına gelir. Kaydetmek, modelinizi paylaşabileceğiniz ve başkalarının da çalışmanızı yeniden oluşturabileceği anlamına gelir. Araştırma modellerini ve tekniklerini yayınlarken çoğu makine öğrenimi uygulayıcısı şunları paylaşır:

* modeli oluşturmak için kod ve
* model için eğitilmiş ağırlıklar veya parametreler

Bu verileri paylaşmak, başkalarının modelin nasıl çalıştığını anlamalarına ve yeni verilerle kendi başlarına denemelerine yardımcı olur.

Caution: Be careful with untrusted code—TensorFlow models are code. See [Using TensorFlow Securely](https://github.com/tensorflow/tensorflow/blob/master/SECURITY.md) for details.

### Seçenekler
Kullandığınız API'ye bağlı olarak TensorFlow modellerini kaydetmenin farklı yolları vardır. Bu kılavuz, TensorFlow'da modeller oluşturmak ve eğitmek için üst düzey bir API olan [tf.keras](https://www.tensorflow.org/guide/keras) kullanır. Diğer yaklaşımlar için, TensorFlow [Save and Restore](https://www.tensorflow.org/guide/saved_model) kılavuzuna veya  [Saving in eager](https://www.tensorflow.org/guide/eager#object-based_saving) konusuna bakın.




## Kurulum
> ## Yüklemeler ve içe aktarmalar
TensorFlow'u ve bağımlılıkları yükleyin ve içe aktarın:

In [3]:
pip install -q pyyaml h5py  # Required to save models in HDF5 format

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.3.0


## Örnek bir veri kümesi alın 
Ağırlıkların nasıl kaydedilip yükleneceğini göstermek için [MNIST dataset](http://yann.lecun.com/exdb/mnist/) veri kümesini kullanacaksınız. Bu çalıştırmaları hızlandırmak için ilk 1000 örneği kullanın:

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

## Bir model tanımlayın 
Basit bir sıralı model oluşturarak başlayın:

In [4]:
# Define a simple sequential model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.metrics.SparseCategoricalAccuracy()])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## Eğitim sırasında kontrol noktalarını kaydedin

Eğitimli bir modeli, onu yeniden eğitmek zorunda kalmadan kullanabilir veya eğitimi kaldığınız yerden alabilirsiniz - eğitim sürecinin kesintiye uğraması durumunda. `tf.keras.callbacks.ModelCheckpoint` geri `tf.keras.callbacks.ModelCheckpoint` , modelin hem eğitim sırasında hem de sonunda sürekli olarak kaydedilmesini sağlar.

### Denetim noktası geri arama kullanımı
Yalnızca eğitim sırasında ağırlık tasarrufu sağlayan bir `tf.keras.callbacks.ModelCheckpoint` geri `tf.keras.callbacks.ModelCheckpoint` oluşturun:

In [5]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])  # Pass callback to training

# Bu, optimize edicinin durumunu kaydetmeyle ilgili uyarılar oluşturabilir.
#Bu uyarılar (ve bu dizüstü bilgisayardaki benzer uyarılar), 
#eski kullanımları caydırmak için mevcuttur ve göz ardı edilebilir.

Epoch 1/10
26/32 [=======================>......] - ETA: 0s - loss: 1.3154 - sparse_categorical_accuracy: 0.6178
Epoch 00001: saving model to training_1\cp.ckpt
32/32 [==============================] - 0s 12ms/step - loss: 1.1940 - sparse_categorical_accuracy: 0.6560 - val_loss: 0.7008 - val_sparse_categorical_accuracy: 0.7940
Epoch 2/10
24/32 [=====================>........] - ETA: 0s - loss: 0.4228 - sparse_categorical_accuracy: 0.8867
Epoch 00002: saving model to training_1\cp.ckpt
32/32 [==============================] - 0s 7ms/step - loss: 0.4131 - sparse_categorical_accuracy: 0.8910 - val_loss: 0.5309 - val_sparse_categorical_accuracy: 0.8370
Epoch 3/10
26/32 [=======================>......] - ETA: 0s - loss: 0.2804 - sparse_categorical_accuracy: 0.9351
Epoch 00003: saving model to training_1\cp.ckpt
32/32 [==============================] - 0s 7ms/step - loss: 0.2764 - sparse_categorical_accuracy: 0.9340 - val_loss: 0.4821 - val_sparse_categorical_accuracy: 0.8390
Epoch 4/10
26/3

Bu, her dönemin sonunda güncellenen tek bir TensorFlow denetim noktası dosyası koleksiyonu oluşturur:

In [37]:
ls {checkpoint_dir}

 Volume in drive C has no label.
 Volume Serial Number is 9899-B9E8

 Directory of C:\Users\MEHMET\jupyter\TENSORFLOW\training_2

27.09.2020  20:07    <DIR>          .
27.09.2020  20:07    <DIR>          ..
27.09.2020  20:07                81 checkpoint
27.09.2020  20:07         1.628.726 cp-0000.ckpt.data-00000-of-00001
27.09.2020  20:07               402 cp-0000.ckpt.index
27.09.2020  20:07         4.886.685 cp-0005.ckpt.data-00000-of-00001
27.09.2020  20:07             1.222 cp-0005.ckpt.index
27.09.2020  20:07         4.886.685 cp-0010.ckpt.data-00000-of-00001
27.09.2020  20:07             1.222 cp-0010.ckpt.index
27.09.2020  20:07         4.886.685 cp-0015.ckpt.data-00000-of-00001
27.09.2020  20:07             1.222 cp-0015.ckpt.index
27.09.2020  20:07         4.886.685 cp-0020.ckpt.data-00000-of-00001
27.09.2020  20:07             1.222 cp-0020.ckpt.index
27.09.2020  20:07         4.886.685 cp-0025.ckpt.data-00000-of-00001
27.09.2020  20:07             1.222 cp-0025.ckpt.index
27

Yeni, eğitimsiz bir model oluşturun. Bir modeli yalnızca ağırlıklardan geri yüklerken, orijinal modelle aynı mimariye sahip bir modeliniz olmalıdır. Aynı model mimarisi olduğundan, modelin farklı bir örneği olmasına rağmen ağırlıkları paylaşabilirsiniz.

Şimdi yeni, eğitimsiz bir modeli yeniden oluşturun ve test setinde değerlendirin. Eğitimsiz bir model şans seviyelerinde performans gösterir (~% 10 doğruluk):

In [9]:
# Create a basic model instance
model = create_model()
model.summary()
# Evaluate the model
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________
32/32 - 0s - loss: 2.3836 - sparse_categorical_accuracy: 0.0470
Untrained model, accuracy:  4.70%


Ardından ağırlıkları kontrol noktasından yükleyin ve yeniden değerlendirin:

In [10]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4151 - sparse_categorical_accuracy: 0.8710
Restored model, accuracy: 87.10%


### Denetim noktası geri arama seçenekleri
Geri arama, denetim noktaları için benzersiz adlar sağlamak ve denetim noktası sıklığını ayarlamak için çeşitli seçenekler sunar.

Yeni bir model eğitin ve her beş dönemde bir benzersiz olarak adlandırılmış kontrol noktalarını kaydedin:

In [11]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=5)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images, 
          train_labels,
          epochs=50, 
          callbacks=[cp_callback],
          validation_data=(test_images,test_labels),
          verbose=0)


Epoch 00005: saving model to training_2\cp-0005.ckpt

Epoch 00010: saving model to training_2\cp-0010.ckpt

Epoch 00015: saving model to training_2\cp-0015.ckpt

Epoch 00020: saving model to training_2\cp-0020.ckpt

Epoch 00025: saving model to training_2\cp-0025.ckpt

Epoch 00030: saving model to training_2\cp-0030.ckpt

Epoch 00035: saving model to training_2\cp-0035.ckpt

Epoch 00040: saving model to training_2\cp-0040.ckpt

Epoch 00045: saving model to training_2\cp-0045.ckpt

Epoch 00050: saving model to training_2\cp-0050.ckpt


Şimdi, ortaya çıkan kontrol noktalarına bakın ve en yenisini seçin:

In [13]:
ls {checkpoint_dir}

 Volume in drive C has no label.
 Volume Serial Number is 9899-B9E8

 Directory of C:\Users\MEHMET\jupyter\TENSORFLOW\training_2

27.09.2020  20:07    <DIR>          .
27.09.2020  20:07    <DIR>          ..
27.09.2020  20:07                81 checkpoint
27.09.2020  20:07         1.628.726 cp-0000.ckpt.data-00000-of-00001
27.09.2020  20:07               402 cp-0000.ckpt.index
27.09.2020  20:07         4.886.685 cp-0005.ckpt.data-00000-of-00001
27.09.2020  20:07             1.222 cp-0005.ckpt.index
27.09.2020  20:07         4.886.685 cp-0010.ckpt.data-00000-of-00001
27.09.2020  20:07             1.222 cp-0010.ckpt.index
27.09.2020  20:07         4.886.685 cp-0015.ckpt.data-00000-of-00001
27.09.2020  20:07             1.222 cp-0015.ckpt.index
27.09.2020  20:07         4.886.685 cp-0020.ckpt.data-00000-of-00001
27.09.2020  20:07             1.222 cp-0020.ckpt.index
27.09.2020  20:07         4.886.685 cp-0025.ckpt.data-00000-of-00001
27.09.2020  20:07             1.222 cp-0025.ckpt.index
27

In [14]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2\\cp-0050.ckpt'

* Not: Varsayılan tensorflow biçimi yalnızca en son 5 denetim noktasını kaydeder.

Test etmek için modeli sıfırlayın ve en son kontrol noktasını yükleyin:

In [15]:
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4882 - sparse_categorical_accuracy: 0.8760
Restored model, accuracy: 87.60%


## Bu dosyalar nedir? 
Yukarıdaki kod, ağırlıkları yalnızca ikili formatta eğitilmiş ağırlıkları içeren [checkpoint](https://www.tensorflow.org/guide/saved_model#save_and_restore_variables) formatlı dosyalar koleksiyonunda depolar. Kontrol noktaları şunları içerir:

* Modelinizin ağırlıklarını içeren bir veya daha fazla parça.
* Hangi kırıkta hangi ağırlıkların saklandığını gösteren bir dizin dosyası.

Bir modeli yalnızca tek bir makinede eğitiyorsanız, son `.data-00000-of-00001` sahip bir parçanız olur: `.data-00000-of-00001`

## Manually save weights

You saw how to load the weights into a model. Manually saving them is just as simple with the `Model.save_weights` method. By default, `tf.keras`—and `save_weights` in particular—uses the TensorFlow [checkpoint](../../guide/checkpoint.ipynb) format with a `.ckpt` extension (saving in [HDF5](https://js.tensorflow.org/tutorials/import-keras.html) with a `.h5` extension is covered in the [Save and serialize models](../../guide/keras/save_and_serialize#weights-only_saving_in_savedmodel_format) guide):

## Ağırlıkları manuel olarak kaydedin
Ağırlıkları bir modele nasıl yükleyeceğinizi gördünüz. `Model.save_weights` yöntemiyle manuel olarak kaydetmek de aynı derecede basittir. Varsayılan, `tf.keras` -ve `save_weights` içinde TensorFlow özellikle-kullanır [checkpoint](../../guide/checkpoint.ipynb) bir ile biçimini `.ckpt` (kaydetme uzantılı[HDF5](https://js.tensorflow.org/tutorials/import-keras.html) bir ile .h5 kaplıdır uzantısı [Save and serialize models](../../guide/keras/save_and_serialize#weights-only_saving_in_savedmodel_format) modelleri rehber):

In [16]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4882 - sparse_categorical_accuracy: 0.8760
Restored model, accuracy: 87.60%


## Save the entire model

Call [`model.save`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#save) to save a model's architecture, weights, and training configuration in a single file/folder. This allows you to export a model so it can be used without access to the original Python code*. Since the optimizer-state is recovered, you can resume training from exactly where you left off.

Entire model can be saved in two different file formats (`SavedModel` and `HDF5`). It is to be noted that TensorFlow `SavedModel` format is the default file format in TF2.x. However, model can be saved in `HDF5` format. More details on saving entire model in the two file formats is described below.

Saving a fully-functional model is very useful—you can load them in TensorFlow.js ([Saved Model](https://www.tensorflow.org/js/tutorials/conversion/import_saved_model), [HDF5](https://www.tensorflow.org/js/tutorials/conversion/import_keras)) and then train and run them in web browsers, or convert them to run on mobile devices using TensorFlow Lite ([Saved Model](https://www.tensorflow.org/lite/convert/python_api#converting_a_savedmodel_), [HDF5](https://www.tensorflow.org/lite/convert/python_api#converting_a_keras_model_))

\*Custom objects (e.g. subclassed models or layers) require special attention when saving and loading. See the **Saving custom objects** section below 




## Tüm modeli kaydedin
Bir modelin mimarisini, ağırlıklarını ve eğitim yapılandırmasını tek bir dosyaya / klasöre kaydetmek için  [`model.save`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#save) çağırın. Bu, bir modeli dışa aktarmanıza olanak tanır, böylece orijinal Python koduna * erişim olmadan kullanılabilir. Optimize edici durumu kurtarıldığından, eğitime tam olarak kaldığınız yerden devam edebilirsiniz.

Tüm model iki farklı dosya biçiminde (`SavedModel` and `HDF5`) kaydedilebilir. TensorFlow `SavedModel` formatının `SavedModel` varsayılan dosya formatı olduğuna dikkat edilmelidir. Ancak model `HDF5` formatında kaydedilebilir. Modelin tamamını iki dosya biçiminde kaydetme hakkında daha fazla ayrıntı aşağıda açıklanmıştır.

Tam işlevli bir modeli kaydetmek çok kullanışlıdır - bunları TensorFlow.js ([Saved Model](https://www.tensorflow.org/js/tutorials/conversion/import_saved_model) ,[HDF5](https://www.tensorflow.org/js/tutorials/conversion/import_keras) ) içine yükleyebilir ve ardından bunları web tarayıcılarında eğitip çalıştırabilir veya TensorFlow Lite ( [Saved Model](https://www.tensorflow.org/lite/convert/python_api#converting_a_savedmodel_) , [HDF5](https://www.tensorflow.org/lite/convert/python_api#converting_a_keras_model_)) kullanarak mobil cihazlarda çalıştırmaya dönüştürebilirsiniz. )

* Özel nesneler (örneğin, alt sınıflı modeller veya katmanlar) kaydederken ve yüklerken özel dikkat gerektirir. Aşağıdaki **Özel nesneleri kaydetme** bölümüne bakın

### SavedModel format

The SavedModel format is another way to serialize models. Models saved in this format can be restored using `tf.keras.models.load_model` and are compatible with TensorFlow Serving. The [SavedModel guide](https://www.tensorflow.org/guide/saved_model) goes into detail about how to serve/inspect the SavedModel. The section below illustrates the steps to saving and restoring the model.

SavedModel biçimi, modelleri serileştirmenin başka bir yoludur. Bu formatta kaydedilen modeller `tf.keras.models.load_model` kullanılarak geri yüklenebilir `tf.keras.models.load_model` ve TensorFlow Sunumu ile uyumludur. [SavedModel guide](https://www.tensorflow.org/guide/saved_model) , SavedModel'in nasıl sunulacağı / inceleneceği hakkında ayrıntılı bilgi verir. Aşağıdaki bölüm, modeli kaydetme ve geri yükleme adımlarını göstermektedir.

In [18]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)



Epoch 1/5
32/32 [==============================] - 0s 4ms/step - loss: 1.1981 - sparse_categorical_accuracy: 0.6540
Epoch 2/5
32/32 [==============================] - 0s 4ms/step - loss: 0.4235 - sparse_categorical_accuracy: 0.8920
Epoch 3/5
32/32 [==============================] - 0s 4ms/step - loss: 0.3006 - sparse_categorical_accuracy: 0.9260
Epoch 4/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2048 - sparse_categorical_accuracy: 0.9500
Epoch 5/5
32/32 [==============================] - 0s 4ms/step - loss: 0.1517 - sparse_categorical_accuracy: 0.9680


In [23]:
# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model') 

A subdirectory or file -p already exists.
Error occurred while processing: -p.
A subdirectory or file saved_model already exists.
Error occurred while processing: saved_model.


INFO:tensorflow:Assets written to: saved_model/my_model\assets


The SavedModel format is a directory containing a protobuf binary and a Tensorflow checkpoint. Inspect the saved model directory:

SavedModel biçimi, bir protobuf ikili dosyası ve bir Tensorflow denetim noktası içeren bir dizindir. Kaydedilen model dizinini inceleyin:

In [47]:
# my_model directory
%ls saved_model

 Volume in drive C has no label.
 Volume Serial Number is 9899-B9E8

 Directory of C:\Users\MEHMET\jupyter\TENSORFLOW\saved_model

27.09.2020  20:28    <DIR>          .
27.09.2020  20:28    <DIR>          ..
27.09.2020  20:28    <DIR>          my_model
               0 File(s)              0 bytes
               3 Dir(s)  300.539.244.544 bytes free


In [53]:
# Contains an assets folder, saved_model.pb, and variables folder.
%cd saved_model/my_model


[WinError 3] Sistem belirtilen yolu bulamıyor: 'saved_model/my_model'
C:\Users\MEHMET\jupyter\TENSORFLOW\saved_model\my_model


In [55]:
%ls 

 Volume in drive C has no label.
 Volume Serial Number is 9899-B9E8

 Directory of C:\Users\MEHMET\jupyter\TENSORFLOW\saved_model\my_model

27.09.2020  20:28    <DIR>          .
27.09.2020  20:28    <DIR>          ..
27.09.2020  20:28    <DIR>          assets
27.09.2020  20:28            78.738 saved_model.pb
27.09.2020  20:28    <DIR>          variables
               1 File(s)         78.738 bytes
               4 Dir(s)  300.538.572.800 bytes free


In [56]:
%quickref #Python Hızlı Referans Kartını görüntüleme

In [58]:
%cd ..

C:\Users\MEHMET\jupyter\TENSORFLOW\saved_model


In [59]:
%cd ..

C:\Users\MEHMET\jupyter\TENSORFLOW


Kaydedilen modelden yeni bir Keras modelini yeniden yükleyin:

In [60]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


Geri yüklenen model, orijinal model ile aynı argümanlarla derlenir. Yüklenen modelle değerlendir ve tahmin et çalıştırmayı deneyin:

In [19]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4378 - sparse_categorical_accuracy: 0.8570
Restored model, accuracy: 85.70%
(1000, 10)


### HDF5 format

Keras, HDF5 standardını kullanarak temel bir kaydetme formatı sağlar. [HDF5](https://en.wikipedia.org/wiki/Hierarchical_Data_Format) standard. 

In [61]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model.h5') 

Epoch 1/5
32/32 [==============================] - 0s 4ms/step - loss: 1.1269 - sparse_categorical_accuracy: 0.6830
Epoch 2/5
32/32 [==============================] - 0s 4ms/step - loss: 0.4204 - sparse_categorical_accuracy: 0.8840
Epoch 3/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2700 - sparse_categorical_accuracy: 0.9360
Epoch 4/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2078 - sparse_categorical_accuracy: 0.9540
Epoch 5/5
32/32 [==============================] - 0s 4ms/step - loss: 0.1509 - sparse_categorical_accuracy: 0.9700


Şimdi, modeli bu dosyadan yeniden oluşturun:

In [62]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


Check its accuracy:

In [63]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

32/32 - 0s - loss: 0.4137 - sparse_categorical_accuracy: 0.8650
Restored model, accuracy: 86.50%


Keras saves models by inspecting the architecture. This technique saves everything:

* The weight values
* The model's architecture
* The model's training configuration(what you passed to compile)
* The optimizer and its state, if any (this enables you to restart training where you left)

Keras is not able to save the `v1.x` optimizers (from `tf.compat.v1.train`) since they aren't compatible with checkpoints. For v1.x optimizers, you need to re-compile the model after loading—losing the state of the optimizer.


Keras, mimariyi inceleyerek modelleri kaydeder. Bu teknik her şeyi kaydeder:

* Ağırlık değerleri
* Modelin mimarisi
* Modelin eğitim yapılandırması (derlemek için geçtiğiniz şey)
* Optimize edici ve varsa durumu (bu, eğitimi kaldığınız yerden yeniden başlatmanıza olanak tanır)

`tf.compat.v1.train` , denetim noktaları ile uyumlu olmadıkları için `v1.x` optimize `v1.x`(`tf.compat.v1.train` ) tf.compat.v1.train . V1.x optimize ediciler için, optimize edicinin durumunu kaybederek yükledikten sonra modeli yeniden derlemeniz gerekir.

### Saving custom objects

If you are using the SavedModel format, you can skip this section. The key difference between HDF5 and SavedModel is that HDF5 uses object configs to save the model architecture, while SavedModel saves the execution graph. Thus, SavedModels are able to save custom objects like subclassed models and custom layers without requiring the orginal code.

To save custom objects to HDF5, you must do the following:

1. Define a `get_config` method in your object, and optionally a `from_config` classmethod.
  * `get_config(self)` returns a JSON-serializable dictionary of parameters needed to recreate the object.
  * `from_config(cls, config)` uses the returned config from `get_config` to create a new object. By default, this function will use the config as initialization kwargs (`return cls(**config)`).
2. Pass the object to the `custom_objects` argument when loading the model. The argument must be a dictionary mapping the string class name to the Python class. E.g. `tf.keras.models.load_model(path, custom_objects={'CustomLayer': CustomLayer})`

See the [Writing layers and models from scratch](https://www.tensorflow.org/guide/keras/custom_layers_and_models) tutorial for examples of custom objects and `get_config`.
